ATTENTION : this workbook needs to run and import the dataset created at the end of this step : [step2](https://www.kaggle.com/leticehs/nlp-bitcoin-analysis-2-preprocessing)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


# 
applying deep learning on twitter’s sentiment analysis

Train Model - use keras to build and train a deep neural network model

Evaluate Model - measure the accuracy of the predictive model, and suggest further improvements

In [ ]:
#1 IMPORTING LIBRARIES AND DATASET

In [ ]:
from time import time
import pandas as pd
import numpy as np
import re
import csv
import matplotlib.pyplot as plt
import seaborn as sns

import itertools
import datetime

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn import decomposition, ensemble
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
import pandas, xgboost, numpy, textblob, string
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers


In [ ]:
#being able to read csv stored in google drive 
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# Reading the dataset with no columns titles and with latin encoding 
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/NLP/tweetsClean.csv')
df.sample(3)

In [ ]:
# Checking if there is any missing value and datatype 
df.info()

In [ ]:

# checking for null values, if any
df.isnull().sum()

In [ ]:
#ditching all row when text is null, as need text for analysis
df.dropna(how='any', inplace=True)
df.sample(3)

2- EXTRACTING FEATURES FROM CLEANED TWEETS 10 min

.1 Count Vectors as features

In [ ]:
#bag of words = OPTION A - need to limit to 1000, else too long and gets 951k results
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=500, stop_words='english',analyzer='word', token_pattern=r'\w{1,}')
bow = bow_vectorizer.fit_transform(df['clean'])
bow.shape


In [ ]:
ditc= bow_vectorizer.vocabulary_
print(ditc)

In [ ]:
print(bow_vectorizer.get_feature_names())

.2 TF-IDF Vectors as features

In [ ]:
#TfIdf = OPTION B - also had to downsize features as too long for not much value added words

tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=500, stop_words='english',analyzer='word', token_pattern=r'\w{1,}')
tfidf = tfidf_vectorizer.fit_transform(df['clean'])
tfidf.shape

In [ ]:
 ditc2 = tfidf_vectorizer.vocabulary_
 print(ditc2)

In [ ]:
print(tfidf_vectorizer.get_feature_names())

In [ ]:
# ngram level tf-idf TAKES 12 minutes to run , after the above that takes 6 min
vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=500)
ngram = vect_ngram.fit_transform(df['clean'])
ngram.shape


.3 Vectors - cannot be used, as words are too small and do not rhyme at all (BTC, ETH, USD...) no dict for that
Word 2 Vec : KeyError: "word 'eth vs btc relative vol spread interesting junction esp given btc dominance v alt season sentiment participants cryptooptions releativevalue' not in vocabulary" or "word 'bizpaye trading platform system unique never done history modern day trade exchanges bizpaye marketplace hodl bartercredit crypto cryptotrading btc onlineshopping merchants ecommerce bb bc retail' not in vocabulary"

3- LAUNCHING MODEL BASES - all above takes 20 min to load

.1 DEFINING X and Y

In [ ]:
#2- CREATING a FAKE Y
#ate 11 dec 2017
#ate 10 dec 2018
#ate end

def senti(x):
  if x < 2018:
    return 'BULL'
  elif x > 2018:
    return 'BULL'
  else:
    return 'BEAR'

df['sent'] = df['year'].apply(lambda x: senti(x) )
df.tail(3)

In [ ]:
df['sent'].value_counts()

In [ ]:
#y = np.array(df['sent']).ravel()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(df['sent'])

.2 LAUNCHING MODEL BASES

In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
    # fit the training dataset on the classifier
    classifier.fit(feature_vector_train, label)
    
    # predict the labels on validation dataset
    predictions = classifier.predict(feature_vector_valid)
    
    if is_neural_net:
        predictions = predictions.argmax(axis=-1)
    
    return metrics.accuracy_score(predictions, y_test), metrics.precision_score(predictions, y_test))

ALL CLASSIFIER MODELS - can start after 13 min of running above

In [ ]:
# Naive Bayes on Count Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), X_count_train, y_train, X_count_test)
print("NB, on Bag of Words : Accuracy, Precision ", accuracy)

# Naive Bayes on Word Level TF IDF Vectors
accuracy = train_model(naive_bayes.MultinomialNB(), X_tfidf_train, y_train, X_tfidf_test)
print("NB, on TF-IDF  : Accuracy, Precision ", accuracy)

In [ ]:
# Log Reg Classifier on Count Vectors - takes 4 min
accuracy = train_model(linear_model.LogisticRegression(), X_count_train, y_train, X_count_test)
print("LR, Bag of Words: : Accuracy, Precision ", accuracy)

# Log Reg Classifier on Word Level TF IDF Vectors
accuracy = train_model(linear_model.LogisticRegression(solver='lbfgs', max_iter=100), X_tfidf_train, y_train, X_tfidf_test)
print("LR, on TF-IDF  : Accuracy, Precision ", accuracy)

In [ ]:
# SVM on Ngram Level TF IDF Vectors - takes forever to calculate MORE THAN 10 MN
accuracy = train_model(svm.SVC(kernel='linear'), X_ngram_train, y_train, X_ngram_test)
print("SVM,  on NGRAM  : Accuracy, Precision, Recall ", accuracy)

In [ ]:
# RF on Count Vectors - takes more than 20 min, never finished
accuracy = train_model(ensemble.RandomForestClassifier(n_estimators = 100),X_count_train, y_train, X_count_test)
print("RF, on Bag of Words : Accuracy, Precision, Recall ", accuracy)

# RF on Word Level TF IDF Vectors
accuracy = train_model(ensemble.RandomForestClassifier(), X_tfidf_train, y_train, X_tfidf_test)
print("RF, on TF-IDF  : Accuracy, Precision, Recall ", accuracy)

In [ ]:
# Extereme Gradient Boosting on Count Vectors - 31 mn for bag of words results and counting
accuracy = train_model(xgboost.XGBClassifier(), X_count_train.tocsc(), y_train, X_count_test.tocsc())
print("Xgb, on Bag of Words : Accuracy, Precision, Recall ", accuracy)

# Extereme Gradient Boosting on Word Level TF IDF Vectors
accuracy = train_model(xgboost.XGBClassifier(), X_tfidf_train.tocsc(), y_train, X_tfidf_test.tocsc())
print("Xgb, on TF-IDF  : Accuracy, Precision, Recall ", accuracy)

In [ ]:
#ON THE MODEL THAT SHOWS MOST ACCURACY, DIG ON ITS SPECS
# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(y_test, y_predict_svm))
# Model Precision: what percentage of positive tuples are labeled as such?
print("Precision:",metrics.precision_score(y_test, y_predict_svm))
# Model Recall: what percentage of positive tuples are labelled as such?
print("Recall:",metrics.recall_score(y_test, y_predict_svm))

THE END ---- TRYING TO CLASSIFY OUR TWEETS INTO A BULL/BEAR SENTIMENT LEADS NO WHERE, I WILL NOW RUN THE CLUSTER WORD ANALYSIS IN THE NEXT NLP4 notebook 